目的
* KerasでCNNを構築しMNISTの画像分類を行う。
* https://keras.io/examples/vision/mnist_convnet/
* https://www.tensorflow.org/tutorials/images/intro_to_cnns?hl=ja

深層学習における基本的実装手順
1. データの読み込み
2. 訓練データと評価データの準備
3. アルゴリズムの選択
4. 学習
5. モデルの評価

準備
* **Graphics Processing Unit (GPU)**を用いて処理を行うために、上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータを**GPU**にしてください。

プログラムで使用するライブラリ/モジュールのインポート

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

データの準備
* MNIST のピクセルの情報は、0〜255 の数値で提供されており、0 が白、255に近くにつれて黒になるような数字になっています。 数値計算の都合上、入力を 0〜1の範囲の数値にした方が良いため、データ型を float に変換したのち、255 で割ります。

NumPy's expand_dims 関数
* 指定された位置に新しい軸を挿入することによって配列の次元を拡張するために使用されます。
* https://pystyle.info/numpy-modify-axis-and-shape/#index_id4
* https://runebook.dev/ja/docs/numpy/reference/generated/numpy.expand_dims
* numpy.expand_dims: https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html

正解データの加工
* MNIST の正解データは、対となる画像に何の数字が書かれているかを表す「0」〜「9」の数字を提供しています。 しかし、ここで作成する人工知能は前述の通り出力が10個の確率になりますので、今のままでは比較が難しいです。
* そこで「0」〜「9」までの数字の形を変換します。 どのように変換するのかといえば、例えば「4」という数字であれば [0,0,0,0,1,0,0,0,0,0] というような、「4」を表す値だけが「1」となるような1次元配列にします。
* keras.utils.to_categorical: ラベルデータの整数値を2値クラスの行列に変換（例「整数 1」を「0,1,0,0,0,0,0,0,0,0」と表現
* keras.utils.to_categorical: https://keras.io/ja/utils/

In [ ]:
# データパラメータ
num_classes = 10
input_shape = (28, 28, 1)

# 1. データの読み込み
# 2. 訓練データと評価データの準備: 訓練セットとテストセットに分ける
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# 画像のピクセルの値を 0~1 の間に正規化
x_train = x_train.astype("float32") / 255 #整数から0～1間の浮動小数に変換
x_test = x_test.astype("float32") / 255 #整数から0～1間の浮動小数に変換
# 画像の形状を確認する（28, 28, 1）
x_train = np.expand_dims(x_train, -1) #新たな次元を追加: 第一引数に元のndarray、第二引数に次元を追加する位置を指定
x_test = np.expand_dims(x_test, -1) #新たな次元を追加: 第一引数に元のndarray、第二引数に次元を追加する位置を指定
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# クラスベクトルを2値クラス行列に変換する
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


モデルの構築 Sequentialモデル: https://www.tensorflow.org/guide/keras/sequential_model?hl=ja
* Sequential モデルは、ニューラルネットワークを構築するための方法の一つです。
* 画像分類の問題では、Sequential モデルを使って、画像を入力とし、それがどのクラスに属するかを出力することができます。
* Keras でモデルを作成するには2つの方法:①Sequential モデル（tf.keras.Sequential）で作成する方法 ②FunctionalAPI（tf.keras.Model）で作成する方法
* モデル作成の流れ: https://zenn.dev/nekoallergy/articles/keras-create-model01

In [ ]:
# 3. アルゴリズムの選択
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5), #何割のノードを不活性化するかを()に数値rateを入れて設定 5割は常にランダムに無視されて学習が進みます
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 10)                  │          16,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,826 (136.04 KB)

 Trainable params: 34,826 (136.04 KB)

 Non-trainable params: 0 (0.00 B)

Output Shape (出力形状)
* バッチサイズ、出力ユニットの個数
* 任意のバッチサイズである場合は、バッチサイズはNone
* https://www.tensorflow.org/tutorials/images/intro_to_cnns?hl=ja

モデルのコンパイル

* モデルの学習を始める前に、**compileメソッドを用いどのような学習処理を行なうかを設定する必要があります**。compileメソッドは**3つの引数**を取ります。生成したmodelインスタンスから**compileメソッド**を呼びだします。
 * **最適化アルゴリズム**: 引数として定義されている最適化手法の識別子を文字列として与える（rmspropやadagradなど）、もしくは Optimizerクラスのインスタンスを与えることができます。
 * **損失関数**: モデルが最小化しようとする目的関数です。引数として定義されている損失関数の識別子を文字列として与える（categorical_crossentropyやmseなど）、もしくは目的関数を関数として与えることができます。
 * **評価関数のリスト**: 分類問題では精度としてmetrics=['accuracy']を指定したくなるでしょう。引数として定義されている評価関数の識別子を文字列として与える、もしくは自分で定義した関数を関数として与えることができます。

学習の実行
* 学習するには、生成したmodelインスタンスから**fit()メソッド**を呼びだします。
* model.fit(入力データ、正解ラベル、バッチサイズ、エポック数）
* バッチサイズを小さくすると消費メモリが少なくて済むが、うまく動かなくなる。
* エポックは学習の繰り返す数

メソッド: **compile**, **fit**, **evaluate**: https://keras.io/api/models/model_training_apis/

In [ ]:
# 4. 学習
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]) #多クラス分類用の交差エントロピー（Categorical Cross Entropy）

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.7630 - loss: 0.7680 - val_accuracy: 0.9770 - val_loss: 0.0822
Epoch 2/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9633 - loss: 0.1214 - val_accuracy: 0.9845 - val_loss: 0.0589
Epoch 3/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9718 - loss: 0.0906 - val_accuracy: 0.9868 - val_loss: 0.0481
Epoch 4/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9776 - loss: 0.0751 - val_accuracy: 0.9885 - val_loss: 0.0420
Epoch 5/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9813 - loss: 0.0635 - val_accuracy: 0.9895 - val_loss: 0.0399
Epoch 6/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9817 - loss: 0.0579 - val_accuracy: 0.9897 - val_loss: 0.0364
Epoch 7/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9835 - loss: 0.0513 - val_accuracy: 0.9902 - val_loss: 0.0367
Epoch 8/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9855 - loss: 0.0475 - val_accuracy: 

モデルを評価するには、生成したmodelインスタンスから**evaluateメソッド**を呼びだします。

In [ ]:
# 5. モデルの評価
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0]) #score[0]にlossを、score[1]にaccuracyを持つリストを返す
print("Test accuracy:", score[1])

Test loss: 0.027086596935987473
Test accuracy: 0.9908000230789185
